In [ ]:
import sys
import os
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
import pandas as pd
import numpy as np
import pickle

from sktime.utils.plotting import plot_series
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon

from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.fbprophet import Prophet
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error

In [ ]:
def data_shaping(symbol_data):
    '''
    yahoo_finance_api2のdataをDataFrameに整形
    '''
    
    # DataFrameに初期化
    df_symbol_data = pd.DataFrame(symbol_data)

    # UNIX時間をUTC時間に変換
    df_symbol_data.timestamp = pd.to_datetime(df_symbol_data.timestamp, unit='ms')
    
    return df_symbol_data

In [ ]:
my_share = share.Share('7974.T') # 東京証券の証券コード指定:任天堂

symbol_data = None

try:
    # 5年分を1日間隔で取得
    symbol_data = my_share.get_historical(share.PERIOD_TYPE_YEAR,
    5,
    share.FREQUENCY_TYPE_DAY,
    1)
except YahooFinanceError as e:
    print(e.message)
    sys.exit(1)

# Dataframeに整形
df_symbol_data = data_shaping(symbol_data)
# 前日までのデータを取得
df_symbol_data = df_symbol_data[:-1]

In [ ]:
# 取引日と終値のみのSeriesを生成
y = df_symbol_data['close']
index = df_symbol_data['timestamp']
y.index = pd.PeriodIndex(index, freq="D")
# 先頭行削除
y = y[1:]
y

In [ ]:
X = df_symbol_data[['open', 'high', 'low', 'volume', 'close']]
X.index = pd.PeriodIndex(index, freq="D")
X

In [ ]:
# 前日の'open', 'high', 'low', 'volume', 'close'を使用して'close'を予測するため一日ずらす
X['open_shift1'] = X['open'].shift(1)
X['high_shift1'] = X['high'].shift(1)
X['low_shift1'] = X['low'].shift(1)
X['volume_shift1'] = X['volume'].shift(1)
X['close_shift1'] = X['close'].shift(1)

# 先頭行削除
X = X[1:]
# 不要な列削除
X = X.drop(['open', 'high', 'low', 'volume', 'close'], axis=1)

X

In [ ]:
# 棒グラフにプロット
plot_series(y, y_label='close')

In [ ]:
# targetの5%をtestデータに分割
y_train, y_test, X_train, X_test = temporal_train_test_split(y, X, test_size=int(len(y) * 0.05) )
# グラフ化
plot_series(y_train.tail(100), y_test.tail(100), labels=["y_train", "y_test"])

In [ ]:
# model用ディレクトリ作成
os.makedirs('./models', exist_ok=True)

# ThetaForecasterモデル
# 季節性の考慮なし
model_name = './models/stock_close_multiple_regression_theta.pkl'

if os.path.exists(model_name):
    print("Already exists model", model_name)
    with open(model_name, 'rb') as f:
        forecaster = pickle.load(f)
else:
    forecaster = ThetaForecaster(sp=12)
    forecaster.fit(y_train, X_train)
    # save model
    with open(model_name,'wb') as f:
        pickle.dump(forecaster,f)
    print("save model", model_name)

In [ ]:
# forecasting horizon
fh = ForecastingHorizon(
    y_test.index, is_relative=False
)
fh

In [ ]:
# 予測
y_pred = forecaster.predict(fh=fh,X=X_test)
# 予測結果のグラフ化
plot_series(y_train.tail(100), y_test, y_pred, labels=["y_train", "y_test", "y_pred"])

In [ ]:
# 精度検証: MAPE(平均絶対パーセント誤差)
mean_absolute_percentage_error(y_test, y_pred)

In [ ]:
# indexをPeriodIndexをDatetimeIndexに変更
y.index = index[1:]
X.index = index[1:]

# targetの5%をtestデータに分割
y_train, y_test, X_train, X_test = temporal_train_test_split(y, X, test_size=int(len(y) * 0.05) )

# Prophetモデル
model_name = './models/stock_close_multiple_regression_prophet.pkl'

if os.path.exists(model_name):
    print("Already exists model", model_name)
    with open(model_name, 'rb') as f:
        forecaster = pickle.load(f)
else:
    forecaster = Prophet(
        seasonality_mode='multiplicative',
        add_country_holidays={'country_name': 'JP'},
        daily_seasonality=True
    )
    forecaster.fit(y_train)
    # save model
    with open(model_name,'wb') as f:
        pickle.dump(forecaster,f)
    print("save model", model_name)

# forecasting horizon
fh = ForecastingHorizon(
    y_test.index, is_relative=False
)

# 予測
y_pred = forecaster.predict(fh=fh, X=X_test)
# 予測結果のグラフ化
plot_series(y_train.tail(100), y_test, y_pred, labels=["y_train", "y_test", "y_pred"])

In [ ]:
# 精度検証: MAPE(平均絶対パーセント誤差)
MAPE = mean_absolute_percentage_error(y_test, y_pred)
# 精度検証: sMAPE(対照的平均絶対パーセント誤差)
SMAPE = mean_absolute_percentage_error(y_test, y_pred, symmetric=True)
print("MAPE:", MAPE, "SMAPE:", SMAPE)